In [1]:
'''
Terminology:
	Episode: one go of the car trying to "live" in the simulation and earn max rewards.
				Episode start from spawning a new car and ends with either car crashing or episode duration limit running out

	Timestep: one frame through simulation: the car gets a obs, reward from prior step and then it makes a decision on control input and sends it to simulation

	Reward logic: each timestep a logic is applied to calculate a reward from latest step. 
				This logic represents you describing the desired behaviour the car needs to learn.

	Policy/model: our objective, what we need learn as part of RL. This is the latest set of rules on what to do at a given camera image.

	Iterations: RL training sessions (multiple episodes and timesteps) when a policy/model is saved. So the policy is changed throughout one iteration
				but then saved in a new file at the end of iteration. This allows to test all models later at different stages of training  
'''	

from stable_baselines3 import PPO #PPO
from stable_baselines3.common.env_checker import check_env

import os
from environment import CarEnv
import time
import carla



In [2]:


# Directory containing saved models
models_dir = "models/2410/1729763726"
logdir = "logs/2410/1729763726"

# Ensure the directories exist
assert os.path.exists(models_dir), f"Model directory {models_dir} does not exist!"
assert os.path.exists(logdir), f"Log directory {logdir} does not exist!"

# List all the saved models in the directory
model_files = [f for f in sorted(os.listdir(models_dir)) if f.endswith('.zip')]
assert len(model_files) > 0, "No models found in the models directory!"


In [3]:

# Load the environment
env = CarEnv()
check_env(env)
env.reset()


2024-10-25 09:33:45,720 - INFO - Starting episode 1
2024-10-25 09:33:45,751 - INFO - Spawned vehicle at: Location(x=111.080017, y=306.419983, z=0.500000)
2024-10-25 09:33:45,753 - INFO - Current waypoint: Location(x=109.659309, y=306.550690, z=0.000000)
2024-10-25 09:33:45,753 - INFO - Target waypoint: Location(x=112.659309, y=306.551697, z=0.000000)
2024-10-25 09:33:45,760 - INFO - lateral_distance: -1.0, speed: 0.0, heading: 0.00010787288968761762
2024-10-25 09:33:45,766 - INFO - Starting episode 2
2024-10-25 09:33:45,770 - INFO - Spawned vehicle at: Location(x=173.870056, y=105.550011, z=0.500000)
2024-10-25 09:33:45,771 - INFO - Current waypoint: Location(x=172.498642, y=105.426109, z=0.000000)
2024-10-25 09:33:45,772 - INFO - Target waypoint: Location(x=169.498642, y=105.428146, z=0.000000)
2024-10-25 09:33:45,778 - INFO - lateral_distance: 1.0, speed: 0.0, heading: 0.9997837490505642
2024-10-25 09:33:45,790 - INFO - Starting episode 3
2024-10-25 09:33:45,802 - INFO - Spawned vehi

Observation: [-1.0000000e+00  0.0000000e+00  1.0787289e-04]
Observation: [1.         0.         0.99978375]
Observation: [1.        0.        0.9988508]


2024-10-25 09:33:46,060 - INFO - Starting episode 4
2024-10-25 09:33:46,078 - INFO - Spawned vehicle at: Location(x=111.080017, y=306.419983, z=0.500000)
2024-10-25 09:33:46,080 - INFO - Current waypoint: Location(x=109.659309, y=306.550690, z=0.000000)
2024-10-25 09:33:46,080 - INFO - Target waypoint: Location(x=112.659309, y=306.551697, z=0.000000)
2024-10-25 09:33:46,105 - INFO - lateral_distance: -1.0, speed: 0.0, heading: 0.00010787288968761762


Vehicle transform: Transform(Location(x=25.530020, y=105.549988, z=0.208296), Rotation(pitch=0.000000, yaw=-179.999634, roll=0.000000)), Yaw:-179.9996337890625, WP_Yaw:179.79315185546875 relative_yaw: 0.20721435546875
Observation: [-0.02494835  0.         -0.00115119]
Observation: [-1.0000000e+00  0.0000000e+00  1.0787289e-04]


(array([-1.0000000e+00,  0.0000000e+00,  1.0787289e-04], dtype=float32), {})

In [4]:

# Select the most recent model to test
latest_model_path = os.path.join(models_dir, model_files[-1])
print(f"Loading model from {latest_model_path}")
model = PPO.load(latest_model_path, env=env)

# Number of episodes to test
num_test_episodes = 5

Loading model from models/2410/1729763726\750000.zip
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:


for episode in range(num_test_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0
    timestep = 0

    print(f"Starting Episode {episode + 1}")

    while not done:
        # Get the action from the model
        action, _states = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward
        timestep += 1
        if done or truncated:
            break

    print(f"Episode {episode + 1} finished. Total Reward: {total_reward} after {timestep} timesteps.\n")

print("Testing completed.")
env.close()


2024-10-25 09:33:48,077 - INFO - Starting episode 5
2024-10-25 09:33:48,081 - INFO - Spawned vehicle at: Location(x=193.779999, y=142.190002, z=0.500000)
2024-10-25 09:33:48,082 - INFO - Current waypoint: Location(x=193.707352, y=142.092743, z=0.000000)
2024-10-25 09:33:48,083 - INFO - Target waypoint: Location(x=193.711441, y=139.092743, z=0.000000)
2024-10-25 09:33:48,090 - INFO - lateral_distance: -1.0, speed: 0.0, heading: -0.49956758287217884


Observation: [-1.          0.         -0.49956757]
Starting Episode 1
Vehicle transform: Transform(Location(x=193.779999, y=142.190002, z=0.208296), Rotation(pitch=0.000000, yaw=-89.999817, roll=0.000000)), Yaw:-89.99981689453125, WP_Yaw:-89.92216491699219 relative_yaw: -0.0776519775390625
Observation: [0.02425507 0.         0.0004314 ]
Vehicle transform: Transform(Location(x=193.783295, y=142.061920, z=0.220926), Rotation(pitch=0.070344, yaw=-89.866646, roll=-0.004730)), Yaw:-89.86664581298828, WP_Yaw:-89.92216491699219 relative_yaw: 0.05551910400390625
Observation: [ 0.0252957   0.1        -0.00030844]
Vehicle transform: Transform(Location(x=193.743439, y=140.851929, z=0.221509), Rotation(pitch=0.028086, yaw=-91.008652, roll=0.001375)), Yaw:-91.00865173339844, WP_Yaw:-89.92216491699219 relative_yaw: -1.08648681640625
Observation: [0.01146248 0.16666667 0.00603604]
Vehicle transform: Transform(Location(x=193.631729, y=139.205475, z=0.221568), Rotation(pitch=0.018722, yaw=-93.239349, r

2024-10-25 09:33:49,945 - INFO - lateral_distance: 0.13136676646854964, speed: 0.31666666666666665, heading: 0.24426006740993925


Vehicle transform: Transform(Location(x=193.927826, y=128.048447, z=0.221878), Rotation(pitch=0.003080, yaw=-85.182793, roll=0.035460)), Yaw:-85.18279266357422, WP_Yaw:-89.92216491699219 relative_yaw: 4.739372253417969
Observation: [0.06712887 0.3        0.14596562]
Action: [2 1], Steer: -0.1, Throttle: 0.2, Brake: 0.0
Vehicle transform: Transform(Location(x=194.124115, y=125.415710, z=0.221830), Rotation(pitch=0.002643, yaw=-85.626831, roll=0.019803)), Yaw:-85.6268310546875, WP_Yaw:-89.92216491699219 relative_yaw: 4.2953338623046875
Observation: [0.13136676 0.31666666 0.24426007]
Vehicle transform: Transform(Location(x=194.114258, y=122.759392, z=0.221833), Rotation(pitch=0.001878, yaw=-90.765701, roll=0.042130)), Yaw:-90.76570129394531, WP_Yaw:-89.92216491699219 relative_yaw: -0.843536376953125
Observation: [0.12687863 0.31666666 0.36553252]
Vehicle transform: Transform(Location(x=193.956467, y=120.102936, z=0.221846), Rotation(pitch=-0.000273, yaw=-93.414856, roll=0.002568)), Yaw:-9

2024-10-25 09:33:50,795 - INFO - Collision detected, resetting vehicle.


...

Vehicle transform: Transform(Location(x=194.772278, y=113.772789, z=0.244993), Rotation(pitch=0.996367, yaw=-90.112350, roll=-1.999573)), Yaw:-90.11235046386719, WP_Yaw:256.4085998535156 relative_yaw: 13.479049682617188
Observation: [0.5103029  0.3        0.34982488]
Next waypoint after collision: Location(x=193.711441, y=139.092743, z=0.000000)
Vehicle transform: Transform(Location(x=193.637070, y=141.291519, z=0.227650), Rotation(pitch=0.256146, yaw=-89.687744, roll=-0.244202)), Yaw:-89.687744140625, WP_Yaw:-89.92216491699219 relative_yaw: 0.2344207763671875
Observation: [-0.02379477  0.11666667 -0.00130234]
Vehicle transform: Transform(Location(x=193.583725, y=139.943924, z=0.221618), Rotation(pitch=0.037962, yaw=-91.447418, roll=-0.059448)), Yaw:-91.44741821289062, WP_Yaw:-89.92216491699219 relative_yaw: -1.5252532958984375
Observation: [-0.04218655  0.18333334  0.00847363]


2024-10-25 09:34:25,350 - INFO - lateral_distance: -0.08157074228720822, speed: 0.23333333333333334, heading: 0.01986024644639757


Action: [6 1], Steer: 0.1, Throttle: 0.2, Brake: 0.0
Vehicle transform: Transform(Location(x=193.468002, y=138.154160, z=0.221542), Rotation(pitch=0.023708, yaw=-93.497009, roll=0.004109)), Yaw:-93.49700927734375, WP_Yaw:-89.92216491699219 relative_yaw: -3.5748443603515625
Observation: [-0.08157074  0.23333333  0.01986025]
Vehicle transform: Transform(Location(x=193.272003, y=135.936401, z=0.221519), Rotation(pitch=0.027157, yaw=-94.546516, roll=-0.011169)), Yaw:-94.54651641845703, WP_Yaw:-89.92216491699219 relative_yaw: -4.624351501464844
Observation: [-0.14790754  0.28333333  0.02569084]
Vehicle transform: Transform(Location(x=193.212570, y=133.558441, z=0.221655), Rotation(pitch=-0.011502, yaw=-89.725700, roll=-0.102386)), Yaw:-89.72570037841797, WP_Yaw:-89.92216491699219 relative_yaw: 0.19646453857421875
Observation: [-0.16879489  0.26666668  0.07484873]
Vehicle transform: Transform(Location(x=193.390411, y=131.169525, z=0.221632), Rotation(pitch=0.007397, yaw=-86.713676, roll=0.01

2024-10-25 09:34:26,986 - INFO - Collision detected, resetting vehicle.
2024-10-25 09:34:27,105 - INFO - Reward: 0.0, Episode time limit reached


Next waypoint after collision: Location(x=193.711441, y=139.092743, z=0.000000)
Action: [2 1], Steer: -0.1, Throttle: 0.2, Brake: 0.0
Vehicle transform: Transform(Location(x=193.672241, y=142.057495, z=0.193551), Rotation(pitch=0.918804, yaw=-89.008934, roll=-1.325623)), Yaw:-89.0089340209961, WP_Yaw:-89.92216491699219 relative_yaw: 0.9132308959960938
Episode 5 finished. Total Reward: 4279.4625529421555 after 400 timesteps.

Testing completed.


In [6]:
env.cleanup() # Clean up the environment
env.reset() # Reset the environment to release the memory

2024-10-25 09:34:27,129 - INFO - Starting episode 10
2024-10-25 09:34:27,133 - INFO - Spawned vehicle at: Location(x=111.080017, y=306.419983, z=0.500000)
2024-10-25 09:34:27,134 - INFO - Current waypoint: Location(x=109.659309, y=306.550690, z=0.000000)
2024-10-25 09:34:27,135 - INFO - Target waypoint: Location(x=112.659309, y=306.551697, z=0.000000)
2024-10-25 09:34:27,146 - INFO - lateral_distance: -1.0, speed: 0.0, heading: 0.00010787288968761762


Observation: [-1.0000000e+00  0.0000000e+00  1.0787289e-04]


(array([-1.0000000e+00,  0.0000000e+00,  1.0787289e-04], dtype=float32), {})

: 